# Import Libraries

In [1]:
import os
import sys
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling



# Import Constants

In [2]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)


In [3]:
from constants import TREECOVER_10_AND_ABOVE, MODIFIED_RASTER, GRUPO_RASTER, DATA_PATH


In [4]:
# Constants
# update the first rasters_to_use_as_mask with the folder with the raster you want to use as a mask
raster_to_use_as_mask =  rasterio.open(TREECOVER_10_AND_ABOVE)
# new output directory
output_dir = os.path.join(DATA_PATH, 'ml_data','tree_masked_rasters')
os.makedirs(output_dir, exist_ok=True)

In [8]:
def treecover_mask(input_mask_ds, input_raster, output_raster_path):
    # Read the data from the mask raster
    mask = input_mask_ds.read(1)

    # Open the second raster
    with rasterio.open(input_raster) as second_ds:
        profile = second_ds.profile
        second_raster = second_ds.read(1)

    # Create a mask where forest is 0 (nonforested)
    mask = mask == 0

    # Apply the mask to the second raster, setting those values to -1
    second_raster[mask] = -1

    # Update the profile for the output raster
    profile.update(
        dtype=rasterio.float32,
        nodata=-1
    )

    # Save the result to a new file
    with rasterio.open(output_raster_path, 'w', **profile) as dst:
        dst.write(second_raster.astype(rasterio.float32), 1)

# Process Files

## Intital Stack

In [9]:
# Mask land use plan
input_lup = GRUPO_RASTER
lup_output_raster = os.path.join(output_dir, 'lup_10_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_lup, lup_output_raster)

C:\Users\bsf31\AppData\Local\Temp\ipykernel_23064\1295753693.py:17: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint16 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  second_raster[mask] = -1


In [13]:
# Mask distance to river raster
input_river = os.path.join(MASKED_RASTERS_DIR[0], 'river_masked.tif')
river_output_raster = os.path.join(output_folder, 'river_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_river, river_output_raster)

In [14]:
# Mask distance to road raster
input_road = os.path.join(MASKED_RASTERS_DIR[0], 'road_masked.tif')
road_output_raster = os.path.join(output_folder, 'road_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_road, road_output_raster)

In [15]:
# Mask soil 
input_soil = os.path.join(MASKED_RASTERS_DIR[0], 'soil_masked.tif')
soil_output_raster = os.path.join(output_folder, 'soil_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_soil, soil_output_raster)

In [ ]:
# Mask precipitation 
input_precipitation = os.path.join(MASKED_RASTERS_DIR[0], 'precipitation_masked.tif')
precipitation_output_raster = os.path.join(output_folder, 'precipitation_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_precipitation, precipitation_output_raster)

In [6]:
# Mask ports
input_ports = os.path.join(MASKED_RASTERS_DIR[0], 'ports_masked.tif')
ports_output_raster = os.path.join(output_folder, 'ports_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_ports, ports_output_raster)

In [7]:
# Mask cities
input_cities = os.path.join(MASKED_RASTERS_DIR[0], 'cities_masked.tif')
cities_output_raster = os.path.join(output_folder, 'cities_treemask.tif')
treecover_mask(raster_to_use_as_mask, input_cities, cities_output_raster)

In [ ]:
# Crop TREECOVER_10_AND_ABOVE to GRUPO_RASTER extent
with rasterio.open(TREECOVER_10_AND_ABOVE) as src:
    with rasterio.open(GRUPO_RASTER) as mask_src:
        # Create a mask for the entire extent of GRUPO_RASTER
        mask = mask_src.dataset_mask()

        # Crop using the mask
        out_image, out_transform = rasterio.mask.mask(src, [mask], crop=True)
        out_meta = src.meta.copy()

# Save the cropped raster as a new file
cropped_raster_path = os.path.join(output_dir, 'cropped_treecover.tif')
out_meta.update({
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})
with rasterio.open(cropped_raster_path, 'w', **out_meta) as dst:
    dst.write(out_image, 1)  # Specify the band index when writing the image
